In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 5)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [ ]:
import numpy as np
import librosa

from composition.instrument import Score, Part, Phrase, show
import composition.common as k
from utils import QuantileTransformer

* violin: mean: -43.36, std: 5.23
* viola: mean: -42.76, std: 7.77
* cello: mean: -57.26, std: 6.12
* flute: mean: -55.27, std: 8.37
* flute2: mean: -55.27, std: 8.37
- Violin 55-103
- Viola 48-91
- Cello 36-76
- Flute 60-96
- Flute2 55-91

In [ ]:
def rescale2(a, mi, ma):
    norm = (a - a[0]) / (a[-1] - a[0])
    return norm * (ma - mi) + mi

In [ ]:
def fractal(points, n=3):
    points = np.array(points)
    segments = points.copy()
    for _ in range(n):
        segments = np.concatenate([rescale2(points, f, s)[:-1] for f, s in zip(segments, segments[1:])])
        segments = np.concatenate((segments, points[-1:]))
    return segments

In [ ]:
def to_delta(d):
    return d[1:] - d[:-1]

def from_delta(d):
    return np.concatenate([[0], np.cumsum(d)])

In [ ]:
def fractal_bpc(values, delta_times, n=0):
    values = fractal(values, n)
    delta_times = to_delta(fractal(from_delta(delta_times), n))
    
    return k.bpc_floor(values, delta_times), delta_times

In [ ]:
def fractal_bpc_gliss(values, delta_times, n=0):
    values = fractal(values, n)
    delta_times = to_delta(fractal(from_delta(delta_times), n))
    
    return k.bpc(values, delta_times), delta_times

In [ ]:
def rescale(a, mi, ma):
    norm = (a - np.min(a)) / (np.max(a) - np.min(a))
    return norm * (ma - mi) + mi

In [ ]:
dur = 16

In [ ]:
seed_p = np.array((0, 0.7, 0.3, 1.0))
seed_a = np.array((0, 0.375, 0.625, 1.))

In [ ]:
show(seed_p, seed_a)

In [ ]:
vln1 = Part('Vln. 1', 'violin')
for i, dr in enumerate([2, 4, 8]):
    seed_d = to_delta(seed_a * dr)
    pitches, durs = fractal_bpc(seed_p, seed_d, i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 66, 66+12)
    # pitches = k.autotune(pitches, 0.95, list(np.random.randint(0, 11, 5)))
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)

    # amps = k.sinusoid(dr, 2/dr, -43.36, 5.23/2)
    amps = np.concatenate([k.triangle(d, 1/d, 0.01) for d in durs])
    # amps += np.random.normal(0, .5, len(amps))
    # amps = k.to_db_loudness(amps, 'violin', min_loudness=-0.1)
    amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
    amps = rescale(amps, -55, -44)
    amps += k.line_segment(dr, -12, 4)
    
    vln1.add_phrase(Phrase(pitches, amps))
    
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
    vln1.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
vln1.show()
vln1.play()

In [ ]:
vln2 = Part('Vln. 2', 'violin')
for i, dr in enumerate([2, 4, 8]):
    seed_d = to_delta(seed_a * dr)
    pitches, durs = fractal_bpc(seed_p[::-1], np.roll(seed_d, 1), i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 61, 61+12)
    # pitches = k.autotune(pitches, 0.95, list(np.random.randint(0, 11, 5)))
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)

    # amps = k.sinusoid(dr, 2/dr, -43.36, 5.23/2)
    amps = np.concatenate([k.triangle(d, 1/d, 0.01) for d in durs])
    # amps += np.random.normal(0, .5, len(amps))
    # amps = k.to_db_loudness(amps, 'violin', min_loudness=-0.1)
    amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
    amps = rescale(amps, -55, -44)
    amps += k.line_segment(dr, -12, 4)
    
    vln2.add_phrase(Phrase(pitches, amps))
    
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
    vln2.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
vln2.show()
vln2.play()

In [ ]:
flt1 = Part('Flt. 1', 'flute')
silence = Phrase(k.constant(17, 66), k.constant(17, -110))
flt1.add_phrase(silence)
for i, dr in enumerate([1, 2, 4]):
    seed_d = to_delta(seed_a * dr)
    pitches, durs = fractal_bpc(np.roll(seed_p, 1), np.roll(seed_d, 2), i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 71, 71+12)
    # pitches = k.autotune(pitches, 0.95, list(np.random.randint(0, 11, 5)))
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)

    # amps = k.sinusoid(dr, 2/dr, -43.36, 5.23/2)
    amps = np.concatenate([k.triangle(d, 1/d, 0.01) for d in durs])
    # amps += np.random.normal(0, .5, len(amps))
    # amps = k.to_db_loudness(amps, 'violin', min_loudness=-0.1)
    amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
    amps = rescale(amps, -55, -54)
    amps += k.line_segment(dr, -8, 0)
    
    flt1.add_phrase(Phrase(pitches, amps))
    flt1.add_phrase(Phrase(pitches[::-1], amps))
    
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
    flt1.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
flt1.show()
flt1.play()

In [ ]:
flt2 = Part('Flt. 2', 'flute2')
silence = Phrase(k.constant(17, 66), k.constant(17, -110))
flt2.add_phrase(silence)
for i, dr in enumerate([1, 2, 4]):
    seed_d = to_delta(seed_a * dr)
    pitches, durs = fractal_bpc(np.roll(seed_p, 2), np.roll(seed_d, 3), i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 66, 66+12)
    # pitches = k.autotune(pitches, 0.95, list(np.random.randint(0, 11, 5)))
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)

    # amps = k.sinusoid(dr, 2/dr, -43.36, 5.23/2)
    amps = np.concatenate([k.triangle(d, 1/d, 0.01) for d in durs])
    # amps += np.random.normal(0, .5, len(amps))
    # amps = k.to_db_loudness(amps, 'violin', min_loudness=-0.1)
    amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
    amps = rescale(amps, -58, -57)
    amps += k.line_segment(dr, -6, 0)
    
    flt2.add_phrase(Phrase(pitches, amps))
    flt2.add_phrase(Phrase(pitches[::-1], amps))
    
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
    flt2.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
flt2.show()
flt2.play()

In [ ]:
# 61 66 68 71

In [ ]:
vlc = Part('Vlc', 'cello')
for i, dr in enumerate([2, 4, 8]):
    seed_d = to_delta(seed_a * dr)
    pitches, durs = fractal_bpc(seed_p, seed_d, i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 42, 42+12)
    # pitches = k.autotune(pitches, 0.95, list(np.random.randint(0, 11, 5)))
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)

    # amps = k.sinusoid(dr, 2/dr, -43.36, 5.23/2)
    amps = np.concatenate([k.triangle(d, 1/d, 0.01) for d in durs])
    # amps += np.random.normal(0, .5, len(amps))
    # amps = k.to_db_loudness(amps, 'violin', min_loudness=-0.1)
    amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
    amps = rescale(amps, -55, -44)
    amps += k.line_segment(dr, -12, 4)
    amps = rescale(amps, -67, -58)
    
    vlc.add_phrase(Phrase(pitches, amps))
    
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
    breath_amp = rescale(breath_amp, -67, -58)
    vlc.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
vlc.show()
vlc.play()

In [ ]:
vla = Part('Vla', 'viola')
for i, dr in enumerate([2, 4, 8]):
    seed_d = to_delta(seed_a * dr)
    pitches, durs = fractal_bpc(seed_p[::-1], np.roll(seed_d, 1), i)
    pitches = np.pad(pitches, (0, k.time_to_step(dr) - len(pitches)), constant_values=pitches[-1])
    pitches = (pitches - min(pitches)) / (max(pitches) - min(pitches))
    pitches = k.rescale(pitches, 49, 49+12)
    # pitches = k.autotune(pitches, 0.95, list(np.random.randint(0, 11, 5)))
    pitches = k.autotune(pitches, 0.95)
    pitches += k.sinusoid(dr, 4, 0, 0.125)

    # amps = k.sinusoid(dr, 2/dr, -43.36, 5.23/2)
    amps = np.concatenate([k.triangle(d, 1/d, 0.01) for d in durs])
    # amps += np.random.normal(0, .5, len(amps))
    # amps = k.to_db_loudness(amps, 'violin', min_loudness=-0.1)
    amps = np.pad(amps, (0, k.time_to_step(dr) - len(amps)), constant_values=amps[-1])
    amps = rescale(amps, -55, -44)
    amps += k.line_segment(dr, -12, 4)
    amps = rescale(amps, -60, -48)
    
    vla.add_phrase(Phrase(pitches, amps))
    
    breathe_p = k.constant(1 + i, pitches[-1])
    vib = k.sinusoid(1+i, 4, 0, 0.125) * (k.phasor(1+i, 1/(1+i)) ** 2)
    breathe_p += vib
    breath_amp = k.phasor(1+i, 1/(1+i)) * 9 - 55
    breath_amp = rescale(breath_amp, -55, -47)
    vla.add_phrase(Phrase(breathe_p, breath_amp))

In [ ]:
vla.show()
vla.play()

In [ ]:
score = Score([vln1, vln2, vla, vlc, flt1, flt2])
# score.show()
# score.play()

In [ ]:
score.save('fractus-duo', './audio-data/december/')

In [ ]:
_, idx = np.unique(flt1.phrases[4].pitch.round(), return_index=True)
flt1.phrases[4].pitch.round()[np.sort(idx)]